The retrieved dataset is too large for the proposed simulation. It is therefore intended reduce it in size. This has been done by limiting the data to the period of the month of December, and the years of 2016 to 2018 inclusive. The number of record points has been reduced to rain, temperature, relative humidity, sun, and visibility. Additionally, the rows of data have been reduced by amalgamating the hourly records into days. The rainfall levels, and hours of sunshine have been added together to provide a total sum for each day. The temperature, relative humidity, and visibility have been averaged for the day in question. This has reduced the number of dataset to 93 (31 x 3) rows, and 6 columns. 

Both the original and new datasets are available in this repository.

### Why was this dataset chosen?

The dataset was chosen for a number of reasons. Primarily, it was chosen as it provides a large volume of data, with interrelated variables. Some of these variables may be positively, or negatively, correlated to each other. This would stand to reason, as the number of hours of sunshine, and the millimetres of rain that have fallen would normally be negatively correlated. Secondly, the dataset is related to the weather in Ireland, or at least Dublin. As the weather .is a favourite topic of conversation, the dataset seemed appropriate.

## Setup
Before the analysis of the dataset can begin, it is necessary to import the data into a dataframe. This will allow the determination of various statistics with regards to the dataset, as well as providing a basis for the simulation to be run. 

The script below will import the data, and set it up in a dataframe.

In [3]:
import numpy as np
import pandas as pd
from datetime import date
import seaborn as sns
import matplotlib.pyplot as plt

url = "https://raw.githubusercontent.com/Clauric/PfDA_Sim_Project/master/Dublin%20Airport%20Weather%202016%20-%202018%20cummulative.csv"

Dublin_df = pd.read_csv(url)

print(Dublin_df.shape)

(93, 6)


In [4]:
print(Dublin_df.corr())
print(Dublin_df.skew())
print(Dublin_df.kurt())

          rain      temp      rhum       sun       vis
rain  1.000000  0.022797  0.186838 -0.183554 -0.363577
temp  0.022797  1.000000  0.012162 -0.432843 -0.272923
rhum  0.186838  0.012162  1.000000 -0.147500 -0.610795
sun  -0.183554 -0.432843 -0.147500  1.000000  0.280341
vis  -0.363577 -0.272923 -0.610795  0.280341  1.000000
rain    2.373325
temp   -0.286573
rhum   -0.289834
sun     1.103147
vis    -0.303844
dtype: float64
rain    5.245557
temp   -0.232388
rhum   -0.558651
sun    -0.166213
vis    -0.512492
dtype: float64
